In [ ]:
#imports para o boot
import sys
import time
import telepot
from telepot.loop import MessageLoop

#imports para o raspador
from urllib.request import urlopen as uReq
import requests
from bs4 import BeautifulSoup as soup
import unidecode


In [ ]:
def getHtml(url):   
    uClient = uReq(url)
    pagina = uClient.read()
    uClient.close()
    return pagina

In [ ]:
def getLinks(pagina):
    links = []
    linksUnicos = []

    page_soup = soup(pagina,"html.parser")

    for link in page_soup.findAll('a'):    
        links.append(link.get('href')) 
    return links

#getLinks(getHtml('https://oglobo.globo.com/'))

In [ ]:
def filterOnlyText(links):
    linksUnicos = []
    for link in links:
        if type(link) == str: 
            linksUnicos.append(link)
            
    return linksUnicos

In [ ]:
def removeDuplicates(links):
    uniqueList = []    
    
    for elem in links:
        if elem not in uniqueList:
            uniqueList.append(elem)    
            
    return uniqueList

In [ ]:
def locateWord(word,site):  
    list = []
    word = unidecode.unidecode(word).lower() #remove os acentos e deixa minúsculo 

    linksUnicos = removeDuplicates(filterOnlyText(getLinks(getHtml(site))))

    for link in linksUnicos:
        temp = link.replace("-"," ")    #substitui os "-" por " "
        text = temp.replace("/"," ")    # remove os "/" por " "
        if 'http' in text:
            if word in text:
                list.append(link)
    return list                

In [ ]:
nivel = 0
jornal = ''
tema = ''
continuar = ''

def handle(msg):
    
    global jornal
    global tema
    global nivel
    global continuar
    
    # dictionary que armazena os sites com seus endereços
    sites = {"g1": "https://g1.globo.com/","uol": "https://www.uol.com.br/",
             "folha": "https://www.folha.uol.com.br/",                         
             "globo":"https://oglobo.globo.com/",
             "piaui":"https://piaui.folha.uol.com.br/",
             "terra":"https://www.terra.com.br/",
             "valor":"https://www.valor.com.br/"
             }
    
    content_type, chat_type, chat_id = telepot.glance(msg)
    
    if nivel == -1:
        if content_type == 'text':             
            nivel = nivel + 1             
    
    if nivel == 0:
        if content_type == 'text':
            bot.sendMessage(chat_id, 'Olá')
            bot.sendMessage(chat_id, 'Escolha um jornal:\nG1\nFolha\nPiaui\nTerra\nValor')                     
            nivel = nivel + 1             
            
    elif nivel == 1:
        if content_type == 'text':          
            jornal = msg['text'] 
            jornal = unidecode.unidecode(jornal).lower()
            jornal = jornal.replace(' ','')
            
            if jornal in sites:
                bot.sendMessage(chat_id, 'Escolha um tema') 
                nivel = nivel + 1
            else:
                bot.sendMessage(chat_id,'Jornal não encontrado em nossa lista, por favor procure outro.')
                nivel = 1
            
    elif nivel == 2:
        if content_type == 'text':            
            tema = msg['text']
            
            for item in locateWord(tema,sites[jornal]):                
                bot.sendMessage(chat_id,item) 
                
            bot.sendMessage(chat_id,"itens encontrados: "+str(len(locateWord(tema,sites[jornal]))))   
            bot.sendMessage(chat_id, 'Deseja procurar outras notícias?')            
            nivel = nivel + 1                                 

    elif nivel == 3:
        if content_type == 'text': 
            continuar = msg['text'] 
            continuar = unidecode.unidecode(continuar).lower()            
            
            if continuar == 'sim':
                nivel = 1
                bot.sendMessage(chat_id, 'Escolha um jornal:\nG1\nFolha\nPiaui\nTerra\nValor')                     
 
            elif continuar == 'nao':
                nivel = -1
                bot.sendMessage(chat_id,"Muito obrigado, até mais.") 
            else:
                bot.sendMessage(chat_id,"Quer continuar? Sim ou não?")
                nivel = 3
        
    
TOKEN = sys.argv[1]  

bot = telepot.Bot('TOKEN!!!!!!!!!!') # Insira seu token aqui
MessageLoop(bot, handle).run_as_thread()
print ('Listening ...')

# Mantém o bot em looping infinito 
while 1:
    time.sleep(10)